In [1]:
#!pip install tensorflow
#!pip install wurlitzer
#!pip install tensorflow_decision_forests
#!pip install imblearn
#!pip install tensorflow_addons

In [2]:
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

import tensorflow_decision_forests as tfdf
import tensorflow_addons as tfa
import tensorflow as tf
import pandas as pd

import seaborn as sns

from collections import Counter
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv

2023-03-13 21:08:47.796080: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 21:08:48.414217: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-13 21:08:49.661060: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/guilherme/cpu-tensorflow-marcelo/nvidia-smi/envs/tf/lib/
2023-03-13 21:08:49.663064: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7';

In [3]:
# y_test     = Array with real values
# yhat_probs = Array with predicted values
def printMetrics(y_test,yhat_probs):
    # predict crisp classes for test set deprecated
    #yhat_classes = model.predict_classes(X_test, verbose=0)
    #yhat_classes = np.argmax(yhat_probs,axis=1)
    yhat_classes = yhat_probs.round()
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_test, yhat_classes)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(y_test, yhat_classes)
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(y_test, yhat_classes)
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, yhat_classes)
    print('F1 score: %f' % f1)
    # kappa
    kappa = cohen_kappa_score(y_test, yhat_classes)
    print('Cohens kappa: %f' % kappa)
    # ROC AUC
    auc = roc_auc_score(y_test, yhat_probs)
    print('ROC AUC: %f' % auc)
    # confusion matrix
    print("\Confusion Matrix")
    matrix = confusion_matrix(y_test, yhat_classes)
    print(matrix)
    
    results = dict()
    results['accuracy'] = accuracy
    results['precision'] = precision
    results['recall'] = recall
    results['f1_score'] = f1
    results['cohen_kappa_score'] = kappa
    results['roc_auc_score'] = auc
    return results

In [4]:
NUMBER_TREES = 500
baseFolder = "../data_2019_processed/"

In [5]:
X_train = pd.read_csv(baseFolder+"train/allData-classification-numeric-normalized.csv")
X_test  = pd.read_csv(baseFolder+"test/allData-classification-numeric-normalized.csv")
X_train_under = pd.read_csv(baseFolder+"train/allData-classification-numeric-normalized_balanced_undersample_nm.csv")
X_train_over = pd.read_csv(baseFolder+"train/allData-classification-numeric-normalized_balanced_oversample_smote.csv")
#X_test  = pd.read_csv(baseFolder+"test/allData-classification-numeric-normalized_balanced_undersample.csv")

#AA = pd.read_csv(baseFolder+"allData-classification-numeric-normalized.csv")

#X_train, X_test = train_test_split(AA,test_size=0.25)
#X_train = pd.read_csv("train_temp.csv")
#X_test = pd.read_csv("test_temp.csv")

In [6]:
X_train

,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class
0,0.00,0.0,0.000869,0.982143,0.000000,1.0,0.0,0.000175,0.0,0.597637,2018-05-15 14:20:45,awake
1,0.00,0.0,0.000869,0.982143,0.000000,1.0,0.0,0.000175,0.0,0.597637,2018-05-15 14:20:45,awake
2,0.25,0.5,0.000869,0.982044,0.604408,1.0,0.0,0.000165,0.0,0.598332,2018-05-15 14:21:15,awake
3,0.25,0.5,0.000869,0.982044,0.604408,1.0,0.0,0.001449,0.0,0.598332,2018-05-15 14:21:45,awake
4,0.25,0.5,0.000869,0.981944,0.601849,1.0,0.0,0.000198,0.0,0.599027,2018-05-15 14:22:15,awake
...,...,...,...,...,...,...,...,...,...,...,...,...
407446,0.25,1.0,0.006924,0.992956,0.644370,1.0,1.0,0.000000,0.0,0.549687,2018-06-12 13:11:39,awake
407447,0.25,1.0,0.006924,0.992956,0.644370,1.0,1.0,0.000000,0.0,0.550382,2018-06-12 13:12:09,awake
407448,0.25,1.0,0.006924,0.992857,0.624127,1.0,1.0,0.000538,0.0,0.551077,2018-06-12 13:13:37,awake
407449,0.00,1.0,0.006924,0.992758,0.540295,0.0,0.0,0.000000,0.0,0.551772,2018-06-12 13:14:07,awake


In [7]:
def transform_output_to_numeric_string(dataframe):
    dataframe['class'] = dataframe['class'].replace(['awake'], '0')
    dataframe['class'] = dataframe['class'].replace(['asleep'], '1')
    
    #dataframe['class'] = dataframe['class'].astype('float32')
    
    return dataframe

selectedColumns=['activity','location','minutes_day',
                  'day_of_week','light','phone_lock',
                  'proximity','sound','time_to_next_alarm','class']

X_train = transform_output_to_numeric_string(X_train[selectedColumns])
X_test = transform_output_to_numeric_string(X_test[selectedColumns])
X_train_over = transform_output_to_numeric_string(X_train_over[selectedColumns])
X_train_under = transform_output_to_numeric_string(X_train_under[selectedColumns])

/tmp/ipykernel_1322/2994769155.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['class'] = dataframe['class'].replace(['awake'], '0')
/tmp/ipykernel_1322/2994769155.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['class'] = dataframe['class'].replace(['asleep'], '1')
/tmp/ipykernel_1322/2994769155.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

In [8]:
# split train and validation datasets
#df_train, df_validation = train_test_split(X_train,test_size=0.25)
# convert data to tensorflow format
#train_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(df_train, label="class")
#validation_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(df_validation, label="class")
train_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(X_train, label="class")
test_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(X_test, label="class")
train_dataset_over = tfdf.keras.pd_dataframe_to_tf_dataset(X_train_over, label="class")
#test_dataset_over = tfdf.keras.pd_dataframe_to_tf_dataset(X_test_over, label="class")
train_dataset_under = tfdf.keras.pd_dataframe_to_tf_dataset(X_train_under, label="class")
#test_dataset_under = tfdf.keras.pd_dataframe_to_tf_dataset(X_test_under, label="class")

/home/guilherme/cpu-tensorflow-marcelo/nvidia-smi/envs/tf/lib/python3.9/site-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  features_dataframe = dataframe.drop(label, 1)
2023-03-13 21:08:55.114293: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-13 21:08:55.178041: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-13 21:08:55.178278: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
20

In [9]:
model = tfdf.keras.RandomForestModel(num_trees = NUMBER_TREES)
modelOver = tfdf.keras.RandomForestModel(num_trees = NUMBER_TREES)
modelUnder = tfdf.keras.RandomForestModel(num_trees = NUMBER_TREES)

model.compile(metrics=[tf.keras.metrics.Accuracy()]) # 
modelOver.compile(metrics=[tf.keras.metrics.Accuracy()])
modelUnder.compile(metrics=[tf.keras.metrics.Accuracy()])
#model.compile(metrics=[tf.keras.metrics.Accuracy(),
#                              tf.keras.metrics.Precision(),
#                              tf.keras.metrics.Recall(),
#                              tfa.metrics.F1Score(num_classes=2,
#                                                  average='macro',
#                                                  threshold=0.5)])

model2 = tfdf.keras.RandomForestModel(num_trees = NUMBER_TREES)
model2.compile(metrics=[tf.keras.metrics.CategoricalAccuracy()])

Use /tmp/tmpsbnvn09n as temporary training directory
Use /tmp/tmpa5ea7zv3 as temporary training directory
Use /tmp/tmptkpionn5 as temporary training directory
Use /tmp/tmp1trrmx_m as temporary training directory


In [10]:
model.fit(train_dataset)
model2.fit(train_dataset)
modelOver.fit(train_dataset_over)
modelUnder.fit(train_dataset_under)

Reading training dataset...
Training dataset read in 0:00:04.020486. Found 407451 examples.
Training model...


[INFO kernel.cc:1176] Loading model from path /tmp/tmpsbnvn09n/model/ with prefix fef99f5d81484f93
[INFO abstract_model.cc:1248] Engine "RandomForestOptPred" built
[INFO kernel.cc:1022] Use fast generic engine


Model trained in 0:00:43.781407
Compiling model...
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.
Reading training dataset...
Training dataset read in 0:00:01.611458. Found 407451 examples.
Training model...


...

Model trained in 0:00:41.504895
Compiling model...
Model compiled.
Reading training dataset...
Training dataset read in 0:00:02.436743. Found 667186 examples.
Training model...


[INFO kernel.cc:1176] Loading model from path /tmp/tmpa5ea7zv3/model/ with prefix 13de0e5a2e2a4587
[INFO abstract_model.cc:1248] Engine "RandomForestOptPred" built
[INFO kernel.cc:1022] Use fast generic engine


Model trained in 0:01:19.454697
Compiling model...
Model compiled.
Reading training dataset...
Training dataset read in 0:00:00.643919. Found 147716 examples.
Training model...


[INFO kernel.cc:1176] Loading model from path /tmp/tmptkpionn5/model/ with prefix 1b39e1c8b4db4b85
[INFO abstract_model.cc:1248] Engine "RandomForestOptPred" built
[INFO kernel.cc:1022] Use fast generic engine


Model trained in 0:00:19.372558
Compiling model...
Model compiled.


In [11]:
print(model.summary())
print(modelUnder.summary())
print(modelOver.summary())

Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (9):
	activity
	day_of_week
	light
	location
	minutes_day
	phone_lock
	proximity
	sound
	time_to_next_alarm

No weights

Variable Importance: MEAN_MIN_DEPTH:
    1.            "__LABEL" 13.641524 ################
    2.           "activity" 10.967490 ############
    3.         "phone_lock"  9.955509 ##########
    4.          "proximity"  8.364260 ########
    5.           "location"  8.279778 ########
    6.        "day_of_week"  6.979696 ######
    7.              "light"  4.825617 ###
    8. "time_to_next_alarm"  3.371709 #
    9.        "minutes_day"  2.445290 
   10.              "sound"  2.089437 

Variable Importance: NUM

_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (9):
	activity
	day_of_week
	light
	location
	minutes_day
	phone_lock
	proximity
	sound
	time_to_next_alarm

No weights

Variable Importance: MEAN_MIN_DEPTH:
    1.            "__LABEL" 13.380137 ################
    2.         "phone_lock" 10.770375 ############
    3.          "proximity"  9.122097 ##########
    4.        "day_of_week"  7.973738 ########
    5.           "location"  7.687287 ########
    6.           "activity"  5.816309 #####
    7.              "light"  4.614142 ###
    8.        "minutes_day"  4.461286 ###
    9. "time_to_next_alarm"  4.311068 ###
   10.              "sound"  1.711481 

Variable Importance: NUM_AS_ROOT:
    1.         

_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (9):
	activity
	day_of_week
	light
	location
	minutes_day
	phone_lock
	proximity
	sound
	time_to_next_alarm

No weights

Variable Importance: MEAN_MIN_DEPTH:
    1.            "__LABEL" 13.777959 ################
    2.           "activity" 10.966332 ############
    3.         "phone_lock" 10.098052 ##########
    4.          "proximity"  8.581871 ########
    5.           "location"  8.053147 ########
    6.        "day_of_week"  6.787495 ######
    7.              "light"  4.812733 ###
    8. "time_to_next_alarm"  3.043593 #
    9.        "minutes_day"  3.003400 #
   10.              "sound"  2.220372 

Variable Importance: NUM_AS_ROOT:
    1.            

In [12]:
results = model.evaluate(test_dataset, return_dict=True)
results2 = model2.evaluate(test_dataset, return_dict=True)
#print("Results: ",results)
resultsOver = modelOver.evaluate(test_dataset, return_dict=True)
resultsUnder = modelUnder.evaluate(test_dataset, return_dict=True)

135/135 [==============================] - 8s 63ms/step - loss: 0.0000e+00 - accuracy: 0.0143


In [13]:
def transform_output_to_numeric(dataframe):
    dataframe['class'] = dataframe['class'].replace(['awake'], '0')
    dataframe['class'] = dataframe['class'].replace(['asleep'], '1')
    
    dataframe['class'] = dataframe['class'].astype('float32')
    
    return dataframe

In [14]:
pred = model.predict(test_dataset)
backup = transform_output_to_numeric(X_test.copy())
printMetrics(backup['class'],pred)

135/135 [==============================] - 7s 48ms/step
Accuracy: 0.821963
Precision: 0.530779
Recall: 0.384205
F1 score: 0.445752
Cohens kappa: 0.342927
ROC AUC: 0.802575
\Confusion Matrix
[[101216   8537]
 [ 15478   9657]]


{'accuracy': 0.821963406678133,
 'precision': 0.5307793778168627,
 'recall': 0.384205291426298,
 'f1_score': 0.44575226753444575,
 'cohen_kappa_score': 0.3429273177592934,
 'roc_auc_score': 0.8025750660609089}

In [15]:
pred1 = model2.predict(test_dataset)
backup1 = transform_output_to_numeric(X_test.copy())
printMetrics(backup1['class'],pred1)

135/135 [==============================] - 7s 48ms/step
Accuracy: 0.821963
Precision: 0.530779
Recall: 0.384205
F1 score: 0.445752
Cohens kappa: 0.342927
ROC AUC: 0.802575
\Confusion Matrix
[[101216   8537]
 [ 15478   9657]]


{'accuracy': 0.821963406678133,
 'precision': 0.5307793778168627,
 'recall': 0.384205291426298,
 'f1_score': 0.44575226753444575,
 'cohen_kappa_score': 0.3429273177592934,
 'roc_auc_score': 0.8025750660609089}

In [16]:
pred2 = modelOver.predict(test_dataset)
backup2 = transform_output_to_numeric(X_test.copy())
printMetrics(backup2['class'],pred2)

135/135 [==============================] - 7s 48ms/step
Accuracy: 0.808916
Precision: 0.490728
Recall: 0.673801
F1 score: 0.567874
Cohens kappa: 0.449077
ROC AUC: 0.801505
\Confusion Matrix
[[92177 17576]
 [ 8199 16936]]


{'accuracy': 0.8089155447482356,
 'precision': 0.490727862772369,
 'recall': 0.673801472050925,
 'f1_score': 0.567874327292236,
 'cohen_kappa_score': 0.449076537555678,
 'roc_auc_score': 0.8015053945090957}

In [17]:
pred3 = modelUnder.predict(test_dataset)
backup3 = transform_output_to_numeric(X_test.copy())
printMetrics(backup3['class'],pred3)

135/135 [==============================] - 8s 61ms/step
Accuracy: 0.622902
Precision: 0.269967
Recall: 0.600716
F1 score: 0.372520
Cohens kappa: 0.155336
ROC AUC: 0.673510
\Confusion Matrix
[[68923 40830]
 [10036 15099]]


{'accuracy': 0.6229019631101358,
 'precision': 0.269967279944215,
 'recall': 0.6007161328824349,
 'f1_score': 0.372520477647291,
 'cohen_kappa_score': 0.15533613693560977,
 'roc_auc_score': 0.6735096461812833}